# Coin Tossing Protocols

Coin tossing is a fundamental cryptographic primitive where two mutually distrusting parties wish to generate a random bit (or string) such that neither party can bias the outcome. This is a key building block for secure computation protocols.

:::{note}
**Reference**: Based on Section 7 of "How To Simulate It" by Yehuda Lindell (2016).
:::

## The Coin Tossing Problem
:label: coin-tossing-problem

### Informal Description

Two parties, $P_1$ and $P_2$, wish to jointly generate a random bit $b \in \{0,1\}$ such that:
- Neither party can bias the outcome (except with negligible probability)
- The outcome is truly random if at least one party is honest
- Both parties learn the outcome

### The Ideal Functionality

In the **ideal world**, there is a trusted party $\mathcal{T}$ that:
1. Receives input from both $P_1$ and $P_2$ (no actual input needed, just notification)
2. Chooses a uniformly random bit $b \leftarrow \{0,1\}$
3. Sends $b$ to both parties

A secure coin tossing protocol must emulate this ideal functionality.

## Protocol 7.1: Single Bit Coin Tossing
:label: single-bit-protocol

We present a simple protocol for tossing a single coin between two parties.

:::{dropdown} **Protocol: Coin Tossing (Single Bit)**

**Inputs**: None (both parties just want a random bit)

**The Protocol**:

1. **$P_1$ commits**:
   - Choose random $r_1 \leftarrow \{0,1\}$
   - Compute commitment $c = \text{Com}(r_1)$ using a commitment scheme
   - Send $c$ to $P_2$

2. **$P_2$ sends random bit**:
   - Choose random $r_2 \leftarrow \{0,1\}$
   - Send $r_2$ to $P_1$

3. **$P_1$ decommits**:
   - Send $r_1$ and decommitment information to $P_2$
   - $P_2$ verifies the decommitment

4. **Both compute output**:
   - Output $b = r_1 \oplus r_2$

:::

### Why This Works

**Intuition**:
- $P_1$ cannot change $r_1$ after seeing $r_2$ (due to commitment binding)
- $P_2$ cannot predict $r_1$ when choosing $r_2$ (due to commitment hiding)
- The XOR ensures that if either bit is random, the output is random

**Formal analysis**:
- If $P_1$ is honest: $r_1$ is truly random and independent of $r_2$, so $b = r_1 \oplus r_2$ is uniformly random
- If $P_2$ is honest: $r_2$ is chosen without knowledge of $r_1$ (by hiding), so $b$ is uniformly random
- If both are honest: $b$ is obviously uniformly random

## Security Against Semi-Honest Adversaries
:label: semi-honest-security

We prove security using the simulation paradigm.

### Simulating a Corrupted $P_1$

:::{dropdown} **Simulator for Corrupted $P_1$**

**Simulator $\mathcal{S}_1$** (when $P_1$ is corrupted):

1. Receive ideal output $b$ from the trusted party
2. Receive commitment $c$ from the adversarial $P_1^*$
3. **Extract**: Use the commitment's extractability to extract $r_1$ from $c$
   - (Note: For semi-honest security, we can just observe what $P_1^*$ sends later)
4. **Compute**: Set $r_2 = b \oplus r_1$ (this ensures the final output will be $b$)
5. Send $r_2$ to $P_1^*$
6. Output whatever $P_1^*$ outputs

**Analysis**:
- In the real protocol, $r_2$ is uniformly random
- In the simulation, $r_2 = b \oplus r_1$ where $b$ is uniformly random
- Since $b$ is random and independent of $r_1$, the value $r_2 = b \oplus r_1$ is also uniformly random
- Therefore, the distributions are identical
:::

### Simulating a Corrupted $P_2$

:::{dropdown} **Simulator for Corrupted $P_2$**

**Simulator $\mathcal{S}_2$** (when $P_2$ is corrupted):

1. Choose random $r_1 \leftarrow \{0,1\}$
2. Compute commitment $c = \text{Com}(r_1)$ and send to $P_2^*$
3. Receive $r_2$ from $P_2^*$
4. **Compute ideal output**: Set $b = r_1 \oplus r_2$ and send to trusted party
5. Send decommitment of $r_1$ to $P_2^*$
6. Output whatever $P_2^*$ outputs

**Analysis**:
- The simulator's view is identically distributed to the real protocol
- The output $b = r_1 \oplus r_2$ in the ideal world matches what would occur in the real protocol
:::

## Protocol 7.3: Multiple Coin Tosses
:label: multiple-coins

To generate $\ell$ random bits efficiently:

:::{note} Protocol: Coin Tossing (Multiple Bits)

**Goal**: Generate $\ell$ random bits $b_1, \ldots, b_\ell$

**The Protocol**:

1. **$P_1$ commits to a string**:
   - Choose random $r_1 \leftarrow \{0,1\}^\ell$
   - Commit to $r_1$: $c = \text{Com}(r_1)$
   - Send $c$ to $P_2$

2. **$P_2$ sends random string**:
   - Choose random $r_2 \leftarrow \{0,1\}^\ell$
   - Send $r_2$ to $P_1$

3. **$P_1$ decommits**:
   - Send $r_1$ and decommitment to $P_2$
   - $P_2$ verifies

4. **Output**:
   - Both parties output $b = r_1 \oplus r_2$ (bitwise XOR)

:::

This is essentially the same as the single-bit protocol but operating on strings.

## The Hybrid Model
:label: hybrid-model

Coin tossing protocols are often analyzed in the **hybrid model**:

### What is the Hybrid Model?

In the hybrid model, the protocol has access to some ideal functionalities as subroutines. For coin tossing:
- We assume an ideal **commitment functionality** $\mathcal{F}_{\text{com}}$
- Parties can commit and decommit by sending messages to $\mathcal{F}_{\text{com}}$
- The functionality guarantees binding and hiding properties perfectly

### Security in the Hybrid Model

We prove:
1. **Coin tossing protocol is secure** in the $\mathcal{F}_{\text{com}}$-hybrid model
2. **Commitment protocol is secure** (instantiates $\mathcal{F}_{\text{com}}$)
3. By **composition theorem**: The combined protocol is secure

This modular approach simplifies security proofs.

## Security Against Malicious Adversaries
:label: malicious-security

The protocol above is secure against **semi-honest** adversaries. What about **malicious** adversaries?

### Attacks by Malicious Adversaries

A malicious $P_1^*$ could:
- Commit to $r_1$, see $r_2$, then refuse to decommit if the outcome is unfavorable
- This allows $P_1^*$ to abort if $r_1 \oplus r_2$ is not the desired value

A malicious $P_2^*$ could:
- In a sequential composition setting, use the commitment $c$ from one session in another session ("commitment reuse attack")

### Preventing Malicious Behavior

To achieve security against malicious adversaries:

1. **Gradual release**: Instead of $P_1$ decommitting all at once, use a gradual release protocol where both parties release bits of their randomness in alternation

2. **Penalties for abort**: Use a protocol where aborting is detectable and punishable

3. **Zero-knowledge proofs**: Have parties prove they are following the protocol correctly

:::{tip} Important Note
Achieving **fair** coin tossing against malicious adversaries (where neither party can bias the outcome even by aborting) is actually impossible without additional assumptions like a common reference string or computational hardness assumptions.
:::

## Simulation Challenges in Coin Tossing
:label: simulation-challenges

Coin tossing presents unique simulation challenges:

### The Output Problem

Unlike in zero-knowledge (where the statement is fixed), in coin tossing:
- The output is **random** and determined by the protocol interaction
- The simulator must produce a specific output (from the ideal functionality)
- But it must do so in a way that looks natural given the adversary's randomness

### The Solution: Programming the Output

The simulator "programs" the protocol to produce the desired output:
- Simulator receives ideal output $b$ from trusted party
- Simulator observes or extracts the adversary's random choice
- Simulator sets its own random choice to force the output to be $b$
- This works because the XOR allows "programming" the result

## Applications of Coin Tossing

Coin tossing is used in:

1. **Secure Computation**: Generating shared randomness for MPC protocols
2. **Leader Election**: Choosing a random party to perform some task
3. **Sampling**: Jointly sampling from complex distributions
4. **Contract Signing**: Fair exchange protocols
5. **Gambling Protocols**: Ensuring fairness in online gambling

## Summary

In this section, we learned:

1. **Coin tossing** allows two parties to generate shared randomness without trusting each other
2. The basic protocol uses **commitment + XOR**: one party commits, the other sends random bits, then XOR determines the output
3. **Simulation** works by "programming" the output: the simulator sets its random choice to force the desired result
4. The **hybrid model** simplifies analysis by assuming ideal commitment functionality
5. **Semi-honest security** is achievable with simple protocols, but **malicious security** requires additional techniques
6. **Fair coin tossing** (where malicious parties cannot bias even by aborting) requires stronger assumptions

Coin tossing demonstrates how simulation proofs handle protocols with random outputs, a key technique for analyzing secure computation protocols.
